<a href="https://colab.research.google.com/github/Fishing-oboro/NLP-learn/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# word2vec (推論ベース)
***
word2vecによって単語の分散表現(ベクトル)を得ることができる。単語をベクトルして扱うことでcos類似度や加算減算への利用が可能になる。
これには、**CBOW**と**skip-gram**という二つの手法がある。二つの手法はどちらも**分布仮説**に基づいており、一つの単語とその周辺の単語の関係性から特徴量を得る。


## CBOW(Continious Bag of Words)
***
この手法では**周辺単語(コンテキスト)から中心単語(ターゲット)**を推測することを目的としたニューラルネットワークを活用する。ニューラルネットワークの推論精度が高まるほど正確な単語分散表現を得ることができる。このCBOWの仕組みについて以下の文章を例に考える。  

　　　　　　　**「私　は　リンゴ　を　食べる　。」**

まず、使用される全単語をもった辞書を作成し、**one-hotベクトル**として扱う。

$$
例：(私→id:0　\begin{matrix}(1&0&0&0&0&0)\end{matrix}, 　リンゴ→id:2　\begin{matrix}(0&0&1&0&0&0)\end{matrix})
$$

次に一つの単語とその周辺単語の組を作成し、それぞれを下記のような正解ラベル、コンテキストとして扱う。

| コンテキスト(入力) | 正解ラベル(出力) | 
| ---- | :----: |
| 私[0], 　　　リンゴ[2] | は[1] |
| は[1], 　　　を[3] | リンゴ[2] |
| リンゴ[2],　食べる[4] | を[3] |
| を[3],　　　。[5] | 食べる[4] |

この表のデータを利用して下記のようなニューラルネットワークを作成し、Winを単語分散表現として得ることができる。このときニューラルネットワークの出力はスコアを示しているが、softMax関数を利用することで各行の重みの和を1とすることができ、正答単語の確率として扱うことができる。この確率と正解ラベルの誤差を小さくすることで推測の精度を高めることができる。
#### NNの構造
---
<div align="center">
<img src="https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/image/CBOW.jpg" width=80%>
</div>

1. 入力層:コンテキストを重みWinを使って低次元のベクトルに変換
2. 中間層:複数のベクトルを結合(和、平均など)
3. 出力層:重みを使って元の次元のベクトルに変換し、softMax関数で確率化。
4. 誤差逆伝搬:正解ラベルと確率の誤差が小さくなるように重みを更新。

#### $W_{in}$がなぜ単語分散表現になるのか
---
私$\begin{matrix}(1&0&0&0&0&0)\end{matrix}$とWinの積は以下のような式になる。  

$$
\left(\begin{matrix}1&0&0&0&0&0\end{matrix}\right)
×
\left(\begin{matrix}
w_{11}&w_{12}&...&w_{1n}\\
w_{21}&w_{22}&...&w_{2n}\\
\\
w_{61}&w_{62}&...&w_{6n}\\
\end{matrix}\right)


=\begin{matrix}(w_{11}&w_{12}&...&w_{1n})\end{matrix}
$$
  

このことからone-hot表現と重み行列の積は単語を示すidの行を取り出すことを意味していることがわかる。したがって、重み行列の各行は辞書に登録されている単語を示す。



## skip-gram
***
この手法ではCBOWとは逆に**中心単語から周辺単語**を推測するニューラルネットワークによって分散表現を得る。このskip-gramについてCBOWで用いた物と同様の例文を用いて考える。

 
　　　　　　　**「私　は　リンゴ　を　食べる　。」**

CBOWと同様に使用される全単語をもった辞書を作成し、one-hotベクトルとして扱う。

$$
例：(私→id:0　\begin{matrix}(1&0&0&0&0&0)\end{matrix}, 　リンゴ→id:2　\begin{matrix}(0&0&1&0&0&0)\end{matrix})
$$

次に一つの単語とその周辺単語の組を作成し、それぞれを下記のような正解ラベル、コンテキストとして扱う。

| 入力 | 正解ラベル(出力) | 
| :----: | ---- |
| は[1]  |私[0], 　　　リンゴ[2] |
| リンゴ[2]  |は[1], 　　　を[3] |
| を[3]  |リンゴ[2],　食べる[4] |
| 食べる[4]  |を[3],　　　。[5] |

この表のデータを利用して下記のようなニューラルネットワークを作成し、Winを単語分散表現として得ることができる。このときニューラルネットワークの出力はスコアを示しているが、softMax関数を利用することで各行の重みの和を1とすることができ、正答単語の確率として扱うことができる。この確率と正解ラベルの誤差を小さくすることで推測の精度を高めることができる。

#### NNの構造
---
<div align="center">
<img src="https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/image/skipgram.p.jpg" width=80%>
</div>

1. 入力層:単語を重みWinを使って低次元のベクトルに変換
2. 中間層:
3. 出力層:重みを使って元の次元のベクトルに変換し、softMax関数で確率化。
4. 誤差逆伝搬:正解ラベルと確率の誤差が小さくなるように重みを更新。

## 実装例
---
例文には青空文庫の[こころ(夏目漱石)](https://www.aozora.gr.jp/cards/000148/card773.html#download)を利用する。

mecabの出力フォーマットのオプションについては[MeCabのコマンドライン引数一覧とその実行例](http://www.mwsoft.jp/programming/munou/mecab_command.html)に詳しく記載されている。

In [ ]:
# mecabに必要なライブラリ取得
!apt-get install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
# テキストの取得
!wget https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/kokoro.txt

In [ ]:
# テキストの加工
with open("./kokoro.txt", mode="r") as f:
    text = f.readlines()
    for line in text:
        if "。" in line:
          with open("./kokoro_new.txt", mode="a") as of:
            of.write(line)

# 基本形で分かち書きにする
!mecab -F"%f[6] " -U"%m " -E"\n" -o kokoro_wakati.txt kokoro_new.txt

Word2Vecの各オプションについては[gemsimのword2vecのためのオプション一覧](https://onemuri.space/note/1ceozcpdt/)に記載されている。

In [ ]:
# モデルの作成
from gensim.models import word2vec
docs = word2vec.LineSentence("kokoro_wakati.txt") 
model = word2vec.Word2Vec(docs,
                        size=100,
                        min_count=1,
                        window=5,
                        iter=3)

In [ ]:
# 類義語の確認
for i in model.most_similar('母'):
    print(i)

## 学習済みモデルの利用
---
word2vecを利用した単語分散表現の取得には、自身で用意した大量の文章データから作成する方法の他に、学習済みモデルを利用する方法がある。
この方法ではmodelの作成にWord2Vec()ではなく、load()やload_word2vec_format()を用いてモデルを読み込む。

#### 学習済みモデルの例
---
- [WikiEntVec 東北大学 乾・鈴木研究室](http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/)  
wikipediaの記事になっている単語とそのリンクを関連付けることで汎用単語と固有単語を見分ける工夫がされている。  
コーパス:Wikipedia、形態素解析:mecab、次元:100~300
- [白ヤギコーポレーション](https://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/)  
モデルが小さいため、比較的軽量。表現力は他モデルにやや劣る。  
コーパス:Wikipedia、形態素解析:mecab、次元:50
- [chiVe(チャイブ) 徳島人工知能NLP研究所 国立国語研究所](https://github.com/WorksApplications/chiVe)    
コーパス:NWJC、形態素解析:Sudachi、次元:300

#### 実装例(白ヤギ)
---

In [ ]:
!wget http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip
!unzip ./latest-ja-word2vec-gensim-model.zip

In [ ]:
from gensim.models import word2vec

model = word2vec.Word2Vec.load('./word2vec.gensim.model')

In [ ]:
for i in model.most_similar('母'):
    print(i)
print('\n')
for i in model.most_similar(positive=['女','美しい'] negative=['']):
    print(i)

### word2vecを使った文章のベクトル化手法の例
---
1. 文章に登場する単語のベクトルの平均を利用する。  
2. TF-IDFと組み合わせる。

